In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
import opendatasets as od

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from CustomDataset import CustomImgDataset

import matplotlib.pyplot as plt


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
#od.download("https://www.kaggle.com/datasets/khoongweihao/covid19-xray-dataset-train-test-sets")

create ds and dl

In [4]:
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128,128)),
    transforms.ToTensor()
])
test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

In [5]:
# preparing the folders 

In [6]:
root_path_train= 'D:/py-py-py-pytorch/Projects/COVID-XRAY/data/train'
root_path_test= 'D:/py-py-py-pytorch/Projects/COVID-XRAY/data/test'

In [7]:
train_dataset = CustomImgDataset(root=root_path_train,transforms=train_transforms)
test_dataset = CustomImgDataset(root=root_path_test,transforms=test_transforms)


In [8]:
print(train_dataset.CLASS_FOLDERS)

['NORMAL', 'PNEUMONIA']


visualize

In [9]:
# it = iter(train_dataset)
# sample_img_tensor = next(it)[0]
# print(sample_img_tensor.shape)
# plt.imshow(sample_img_tensor.squeeze())

In [10]:
train_dataloader = DataLoader(train_dataset,batch_size=16,shuffle=True)

In [11]:
loader_it = iter(train_dataloader)
sample_batch = next(loader_it)
sample_batch

[tensor([[[[0.0627, 0.0667, 0.0706,  ..., 0.0784, 0.0784, 0.0784],
           [0.0588, 0.0667, 0.0706,  ..., 0.0784, 0.0784, 0.0784],
           [0.0588, 0.0667, 0.0706,  ..., 0.0784, 0.0784, 0.0784],
           ...,
           [0.1294, 0.1059, 0.0941,  ..., 0.0706, 0.0745, 0.0745],
           [0.1333, 0.1098, 0.1020,  ..., 0.0784, 0.0824, 0.0824],
           [0.1373, 0.1137, 0.1059,  ..., 0.1020, 0.1059, 0.1098]]],
 
 
         [[[0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0275,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0235,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0235,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0078, 0.0039, 0.0039,  ..., 0.1686, 0.1412, 0.1137],
           [0.0078, 0.0000, 0.0000,  ..., 0.1647, 0.1412, 0.1098],
           [0.0078

In [12]:
sample_features,sample_label = sample_batch

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(32,16,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            
            nn.Flatten(),
            
            nn.Linear(4096,128),
            nn.ReLU(),

            nn.Linear(128,2)
        )
    
    def forward(self,inputs):
        out = self.network(inputs)
        return torch.softmax(out,dim=-1)
        


In [14]:
covid_model = NeuralNetwork().to(device)
loss_function = nn.CrossEntropyLoss()
optimizier = torch.optim.Adam(covid_model.parameters(),lr=3e-4)

Training the model

In [15]:
num_epochs = 100
losses = []
def fit_model(num_epochs):
    for epoch in range(num_epochs):
        print("Epoch {}".format({epoch+1}))
        curr_loss = []
        for idx,batch in enumerate(iter(train_dataloader)):
            x,y = batch

            covid_model.train()
            pred = covid_model(x.to(device))
            loss = loss_function(pred.to(device),y.to(device))
            
            loss.backward()

            optimizier.step()
            optimizier.zero_grad()

            curr_loss.append(loss)
        
        mean_epoch_loss = torch.tensor(curr_loss).mean()
        losses.append(mean_epoch_loss)
        print("Loss = {}".format(mean_epoch_loss))


In [16]:
fit_model(1)

Epoch {1}
Loss = 0.6608047485351562


In [17]:
# epoch = np.arange(num_epochs)+1
# plt.figure(figsize=(5,5))
# plt.subplot(121)
# plt.title("Loss vs Epochs")
# plt.plot(epoch,losses,label="Training Loss")
# plt.tight_layout()
# plt.legend()
# plt.show()

In [18]:
# it = iter(test_dataset)
# plt.imshow(next(it)[0].squeeze())

In [19]:
test_dataloader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [20]:
with torch.no_grad():
    correct = 0
    for img,labels in test_dataloader:
        pred = covid_model(img.to(device))
        print(pred.shape, labels.shape)
        for i in range(len(pred)):
            if(pred[i].max() == pred[i][labels[i].item()]):
                correct += 1

    acc = correct/len(test_dataloader)

    print(acc)

torch.Size([16, 2]) torch.Size([16])
torch.Size([16, 2]) torch.Size([16])
torch.Size([3, 2]) torch.Size([3])
6.666666666666667
